In [12]:
import pandas as pd
import numpy as np
r_data = pd.read_csv('archive2/ratings.csv')
m_data = pd.read_csv('archive2/movies.csv')

In [6]:
u_m_rating = r_data.pivot(
    index = 'userId',
    columns = 'movieId',
    values = 'rating'
).fillna(0)

In [29]:
matrix = u_m_rating.values

#user mean
user_mean = np.mean(matrix,axis = 1)

matrix_u_mean = matrix-user_mean.reshape(-1,1)

In [34]:
matrix_u_mean

array([[-0.00562541, -0.00562541, -0.00562541, ..., -0.00562541,
        -0.00562541, -0.00562541],
       [-0.02923009, -0.02923009, -0.02923009, ..., -0.02923009,
        -0.02923009, -0.02923009],
       [-0.02007501, -0.02007501, -0.02007501, ..., -0.02007501,
        -0.02007501, -0.02007501],
       ...,
       [-0.01367748, -0.01367748, -0.01367748, ..., -0.01367748,
        -0.01367748, -0.01367748],
       [ 3.98698434, -0.01301566, -0.01301566, ..., -0.01301566,
        -0.01301566, -0.01301566],
       [ 4.95030885, -0.04969115, -0.04969115, ..., -0.04969115,
        -0.04969115, -0.04969115]])

In [36]:
pd.DataFrame(matrix_u_mean, columns = u_m_rating.columns).head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
0,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,...,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625
1,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,3.970770,...,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230
2,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,...,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075
3,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,3.902162,...,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838
4,-0.043128,-0.043128,3.956872,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,...,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128


In [45]:
from numpy.linalg import svd
from scipy.sparse.linalg import svds

In [49]:
U, sigma, Vt = svds(matrix_u_mean , k = 12)

In [50]:
U.shape, sigma.shape, Vt.shape

((671, 12), (12,), (12, 9066))

In [52]:
#0이 포함된 대칭 행렬 변환
sigma = np.diag(sigma)

In [55]:
svd_user = np.dot(np.dot(U,sigma),Vt)+user_mean.reshape(-1,1)

In [57]:
svd_pred = pd.DataFrame(svd_user, columns = u_m_rating.columns)

In [70]:
def recommand(svd_pred,u_id, ori_movies,ori_ratings,num_recommendation = 5):
    
    user_idx = u_id -1
    print("A",user_idx)
    sorted_u_predictions = svd_pred.iloc[user_idx].sort_values(ascending = False)
    print("B",sorted_u_predictions)
    
    user_data = ori_ratings[ori_ratings.userId == u_id]
    print("C",user_data)
    user_history = user_data.merge(ori_movies, on = 'movieId').sort_values(['rating'],ascending = False)
    print("D",user_history)
    
    #없는거 표시
    recommendation = ori_movies[~ori_movies['movieId'].isin(user_history['movieId'])]
    print("F",recommendation)
    recommendation = recommendation.merge(pd.DataFrame(sorted_u_predictions).reset_index(), on = 'movieId')
    print("G",recommendation)
    recommendation = recommendation.rename(columns = {user_idx:'Predictions'}).sort_values('Predictions',ascending = False)
    print("H",recommendation)
    
    

In [68]:
r_data.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [71]:
recommand(svd_pred,330,m_data,r_data,10)

A 329
B movieId
318     2.078816
296     2.060064
593     1.927825
356     1.772814
527     1.663605
          ...   
2140   -0.148684
7373   -0.150847
1215   -0.175392
2005   -0.219911
3033   -0.221206
Name: 329, Length: 9066, dtype: float64
C        userId  movieId  rating  timestamp
45916     330       16     3.0  948574059
45917     330       22     3.0  948576623
45918     330       25     3.0  948577275
45919     330       32     4.0  948578017
45920     330       41     5.0  948577200
...       ...      ...     ...        ...
46006     330     3100     4.0  948577933
46007     330     3246     4.0  948577543
46008     330     3256     3.0  948576674
46009     330     3258     2.0  948575129
46010     330     3260     4.0  948577232

[95 rows x 4 columns]
D     userId  movieId  rating  timestamp  \
47     330     1094     5.0  948577200   
14     330      213     5.0  948577039   
34     330      527     5.0  948577164   
45     330     1035     5.0  948574126   
48     330     1